In [6]:


!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client markdown

import re
import markdown
from google.colab import auth
from googleapiclient.discovery import build
import google.auth

# Set up the google documents apis
auth.authenticate_user()

SCOPES = ['https://www.googleapis.com/auth/documents']
creds, _ = google.auth.default(scopes=SCOPES)
docs_service = build('docs', 'v1', credentials=creds)

# reading the data from below markdown text

MARKDOWN_CONTENT = r'''
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
'''


In [7]:

# Parsing markdown to generate the google docs requests

def parse_markdown_body_and_footer(md_text):
    """
    Splits the markdown content into two parts:
    - Everything before the line '---' (the main body)
    - Everything after '---' (the footer lines)
    Returns (body_text, footer_text).
    """
    # Split on the first occurrence of '---'
    if '---' in md_text:
        parts = md_text.split('---', 1)
        body = parts[0].strip()
        footer = parts[1].strip()
        return body, footer
    else:
        return md_text, ""  # No footer detected

def build_footer_requests(footer_text, insertion_index):
    """
    Takes the footer text and applies a distinct style
    Returns a list of requests to insert the footer lines at the given insertion_index.
    """
    if not footer_text:
        return [], insertion_index

    lines = footer_text.splitlines()
    requests = []

    for line in lines:
        # Skip empty lines
        if not line.strip():
            continue

        # Insert the line
        insert_req = {
            'insertText': {
                'location': {'index': insertion_index},
                'text': line + '\n'
            }
        }
        requests.append(insert_req)
        text_style_req = {
            'updateTextStyle': {
                'range': {
                    'startIndex': insertion_index,
                    'endIndex': insertion_index + len(line)
                },
                'textStyle': {
                    'italic': True,
                    'fontSize': {
                        'magnitude': 10,
                        'unit': 'PT'
                    },
                    'foregroundColor': {
                        'color': {
                            'rgbColor': {
                                'red': 0.4,  # Turning the footer elements in light gray
                                'green': 0.4,
                                'blue': 0.4
                            }
                        }
                    }
                },
                'fields': 'italic,fontSize,foregroundColor'
            }
        }
        requests.append(text_style_req)

        # Move insertion_index forward
        insertion_index += len(line) + 1

    return requests, insertion_index

def parse_markdown_to_requests(md_text):
    """
    Parse the main body of markdown (headings, bullets, checkboxes, mentions)
    and build Google Docs insert/style requests.
    """
    lines = md_text.strip().split('\n')
    requests = []
    insertion_index = 1

    def insert_paragraph(text, style, insertion_index, bold_mentions=False):
        """
        Helper to insert a paragraph with the specified style.
        If bold_mentions=True, we detect '@...' patterns and bold/color them.
        """
        if not bold_mentions or '@' not in text:
            # Simple insertion
            return {
                'insertText': {
                    'location': {'index': insertion_index},
                    'text': text + '\n'
                }
            }, {
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': insertion_index,
                        'endIndex': insertion_index + len(text) + 1
                    },
                    'paragraphStyle': {
                        'namedStyleType': style
                    },
                    'fields': 'namedStyleType'
                }
            }
        else:
            # We need multiple requests to style mentions
            main_insert = {
                'insertText': {
                    'location': {'index': insertion_index},
                    'text': text + '\n'
                }
            }
            paragraph_style_req = {
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': insertion_index,
                        'endIndex': insertion_index + len(text) + 1
                    },
                    'paragraphStyle': {
                        'namedStyleType': style
                    },
                    'fields': 'namedStyleType'
                }
            }

            # Build mention bold/color requests
            text_style_reqs = []
            pattern = r'(@[\w]+)'
            base_pos = insertion_index
            for match in re.finditer(pattern, text):
                start = base_pos + match.start()
                end = base_pos + match.end()
                text_style_reqs.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': start,
                            'endIndex': end
                        },
                        'textStyle': {
                            'bold': True,
                            'foregroundColor': {
                                'color': {
                                    'rgbColor': {'red': 0.0, 'green': 0.0, 'blue': 1.0}
                                }
                            }
                        },
                        'fields': 'bold,foregroundColor'
                    }
                })

            return main_insert, paragraph_style_req, text_style_reqs

    # Go line by line
    for line in lines:
        raw = line.strip()
        if not raw:
            continue

        # Detect headings
        if raw.startswith('# '):
            text = raw[2:]
            reqs = insert_paragraph(text, 'HEADING_1', insertion_index, bold_mentions=False)
        elif raw.startswith('## '):
            text = raw[3:]
            reqs = insert_paragraph(text, 'HEADING_2', insertion_index, bold_mentions=False)
        elif raw.startswith('### '):
            text = raw[4:]
            reqs = insert_paragraph(text, 'HEADING_3', insertion_index, bold_mentions=False)
        # Detect bullet items
        elif re.match(r'^[-*]\s', raw):
            # Check for checkbox pattern
            checkbox_match = re.match(r'^[-*]\s\[( |x)\]\s(.*)', raw)
            if checkbox_match:
                # It's a checkbox bullet
                checked = (checkbox_match.group(1) == 'x')
                text_after_checkbox = checkbox_match.group(2)

                # Insert text
                main_insert = {
                    'insertText': {
                        'location': {'index': insertion_index},
                        'text': text_after_checkbox + '\n'
                    }
                }
                requests.append(main_insert)

                # Style any mentions
                mention_pattern = r'(@[\w]+)'
                for mention_match in re.finditer(mention_pattern, text_after_checkbox):
                    start_pos = insertion_index + mention_match.start()
                    end_pos = insertion_index + mention_match.end()
                    requests.append({
                        'updateTextStyle': {
                            'range': {
                                'startIndex': start_pos,
                                'endIndex': end_pos
                            },
                            'textStyle': {
                                'bold': True,
                                'foregroundColor': {
                                    'color': {
                                        'rgbColor': {'red': 0.0, 'green': 0.0, 'blue': 1.0}
                                    }
                                }
                            },
                            'fields': 'bold,foregroundColor'
                        }
                    })

                # Advance index
                insertion_index += len(text_after_checkbox) + 1

                # Convert paragraph into checkbox bullet
                requests.append({
                    'createParagraphBullets': {
                        'range': {
                            'startIndex': insertion_index - (len(text_after_checkbox) + 1),
                            'endIndex': insertion_index
                        },
                        'bulletPreset': 'BULLET_CHECKBOX'
                    }
                })
                continue
            else:
                # Normal bullet
                text = raw[2:].strip()
                reqs = insert_paragraph(text, 'NORMAL_TEXT', insertion_index, bold_mentions=True)
        else:
            # Normal paragraph
            reqs = insert_paragraph(raw, 'NORMAL_TEXT', insertion_index, bold_mentions=True)

        # Handle requests (flatten tuple)
        if isinstance(reqs, tuple):
            if len(reqs) == 3:
                requests.append(reqs[0])        # insertText
                requests.append(reqs[1])        # updateParagraphStyle
                requests.extend(reqs[2])        # mention textStyles
                inserted_text = reqs[0]['insertText']['text']
            else:
                requests.append(reqs[0])
                requests.append(reqs[1])
                inserted_text = reqs[0]['insertText']['text']
        else:
            # If something returns a single request, handle that (unlikely in this snippet)
            requests.append(reqs)
            inserted_text = reqs['insertText']['text']

        # Increase insertion index by the length of text plus newline
        insertion_index += len(inserted_text)

    return requests, insertion_index



In [8]:
# create a new doc and fill body and footer for the google docs

def create_and_format_doc(md_text):
    """
    Creating a new Google Doc
    Parsing the markdown into main body (before '---') and footer (after '---')
    Send batch requests to fill the doc
    """
    # Create a new empty doc
    title = "Automated Meeting Notes"
    doc = docs_service.documents().create(body={"title": title}).execute()
    doc_id = doc.get('documentId')
    print(f"Created document with title: {title}\nDocument ID: {doc_id}")

    # Split markdown into body & footer parts
    body_md, footer_md = parse_markdown_body_and_footer(md_text)

    # Build requests for the body
    body_requests, end_index = parse_markdown_to_requests(body_md)

    # Build requests for the footer, using the next insertion index
    footer_requests, _ = build_footer_requests(footer_md, insertion_index=end_index)

    # Combine
    all_requests = body_requests + footer_requests

    if all_requests:
        docs_service.documents().batchUpdate(
            documentId=doc_id,
            body={'requests': all_requests}
        ).execute()
        print("Document updated successfully.")
    else:
        print("No requests generated; document remains blank.")

    return doc_id



In [9]:
# Execute to generate the text file
if __name__ == "__main__":
    doc_id = create_and_format_doc(MARKDOWN_CONTENT)
    print(f"View your doc here: https://docs.google.com/document/d/{doc_id}/edit")

Created document with title: Automated Meeting Notes
Document ID: 1p-2Y9VLOGuBIhsqdBudJBFVf7ZT-CIg-Jp5nP9476rE
Document updated successfully.
View your doc here: https://docs.google.com/document/d/1p-2Y9VLOGuBIhsqdBudJBFVf7ZT-CIg-Jp5nP9476rE/edit
